In [1]:
import sys

In [2]:
sys.path.insert(0, "..")

In [3]:
import spacy
import medspacy

# Overview
The previous notebooks showed the components which are loaded by a default medSpaCy model: A custom tokenizer; sentence splitter; target matcher; and context. MedSpaCy also provides other components which can be instantiated and added to an existing pipeline:
- `Sectionizer`: Detecting section boundries in a clinical note
- `Preprocessor`: Destructive preprocessing for simplifying or cleaning up text
- `Postprocessor`: Additional business logic for altering or removing entities at the end of pipeline processing

We'll start by loading some of the custom rules shown in previous notebooks:

In [4]:
nlp = medspacy.load()

/Users/alecchapman/opt/anaconda3/envs/medspacy-37/lib/python3.7/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
from medspacy.ner import TargetRule

In [6]:
target_matcher = nlp.get_pipe("target_matcher")

In [7]:
target_rules = [
    TargetRule(literal="abdominal pain", category="PROBLEM"),
    TargetRule("stroke", "PROBLEM"),
    TargetRule("hemicolectomy", "TREATMENT"),
    TargetRule("Hydrochlorothiazide", "TREATMENT"),
    TargetRule("colon cancer", "PROBLEM"),
    TargetRule("radiotherapy", "PROBLEM",
              pattern=[{"LOWER": "xrt"}]),
    TargetRule("metastasis", "PROBLEM"),
    
]

In [8]:
target_matcher.add(target_rules)

In [9]:
with open("./discharge_summary.txt") as f:
    text = f.read()

# Section detection
We are often interested in which section of a clinical note an entity occurs in. This can be useful for setting attributes like temporality (similar to ConText) or for extracting entities from specific sections of the note.

MedSpaCy includes the `Sectionizer` class from the [clinical_sectionizer](https://github.com/medspacy/sectionizer) package. Similar to `ConTextComponent`, we can instantiate this with default rules and add new ones to fit our specific data. Section detection is especially dependent on your data, as each EHR will use different note formatting.

In [10]:
from medspacy.section_detection import Sectionizer

In [11]:
sectionizer = Sectionizer(nlp, patterns="default")

In [12]:
nlp.add_pipe(sectionizer)

In [13]:
doc = nlp(text)

`visualize_ent` will now highlight section titles in addition to entities and context modifiers:

In [14]:
from medspacy.visualization import visualize_ent

In [15]:
visualize_ent(doc)

We can see here that the default rules did not catch the section title **"Brief Hospital Course"**. We can add a pattern to our sectionizer by passing in a dictionary with two key/pair values:
- **"section_title"**: The normalized section title
- **"pattern"**: A spaCy pattern to match the text (either a string or a list of Token dictionaries, similar to other components)

In [16]:
section_patterns = [
    {"section_title": "hospital_course", "pattern": "Brief Hospital Course:"}
]

In [17]:
sectionizer.add(section_patterns)

In [18]:
visualize_ent(nlp("""
Brief Hospital Course:
Ms. [**Known patient lastname 2004**] was admitted on [**2573-5-30**]. Ultrasound at the time of
admission demonstrated pancreatic duct dilitation and an
edematous gallbladder. She was admitted to the ICU.
"""))

The sectionizer will add attributes to allow us to access section data. The `doc` object will have these 3 attributes:

In [19]:
# Normalized section titles
print(doc._.section_titles)

[None, 'other', 'allergies', 'chief_complaint', 'history_of_present_illness', 'past_medical_history', 'social_history', 'family_history', 'hospital_course', 'medications', 'observation_and_plan', 'patient_instructions', 'signature']


In [20]:
# The Spans of the doc representing section headers
doc._.section_headers

[None,
 Service:,
 Allergies:,
 Chief Complaint:,
 History of Present Illness:,
 Past Medical History:,
 Social History:,
 Family History:,
 Brief Hospital Course:,
 Discharge Medications:,
 Discharge Diagnosis:,
 Discharge Instructions:,
 Signed electronically by:]

In [21]:
# Spans of the actual sections of the notes
doc._.section_spans[:5]

[Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]
 
 Date of Birth:  [**2498-8-19**]             Sex:   F
 ,
 Service: SURGERY
 ,
 Allergies:
 Hydrochlorothiazide
 
 Attending:[**First Name3 (LF) 1893**],
 Chief Complaint:
 Abdominal pain
 
 Major Surgical or Invasive Procedure:
 PICC line [**6-25**]
 ERCP w/ sphincterotomy [**5-31**]
 
 ,
 History of Present Illness:
 74y female with type 2 dm and a recent stroke affecting her
 speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis.
 ]

Which can be zipped up as tuples in one attribute:

In [22]:
print(doc._.sections[:5])

[Section(section_title=None, section_header=None, section_parent=None, section_span=Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

), Section(section_title='other', section_header=Service:, section_parent=None, section_span=Service: SURGERY

), Section(section_title='allergies', section_header=Allergies:, section_parent=None, section_span=Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
), Section(section_title='chief_complaint', section_header=Chief Complaint:, section_parent=None, section_span=Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


), Section(section_title='history_of_present_illness', section_header=History of Present Illness:, section_parent=None, section_span=History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdomi

For each section detected in the note, a namedtuple containing these 4 attributes is created: we'll print out the
- The normalized section title
- The actual span of the doc containing the section header
- The parent section
- The entire span of the section in the doc
These are explained in more detail in the `section_detection/` notebooks.

In [23]:
for section in doc._.sections:
    print(section.section_title, section.section_header, section.section_parent)
    print(section[:25])
    print("----------------")

None None None
(None, None, None, Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

)
----------------
other Service: None
('other', Service:, None, Service: SURGERY

)
----------------
allergies Allergies: None
('allergies', Allergies:, None, Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
)
----------------
chief_complaint Chief Complaint: None
('chief_complaint', Chief Complaint:, None, Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


)
----------------
history_of_present_illness History of Present Illness: None
('history_of_present_illness', History of Present Illness:, None, History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis.

)
----------------
past_medical_history Past M

Each entity has similar attributes:

In [24]:
for ent in doc.ents:
    print(ent, ent._.section_title, ent._.section_header, ent._.section_span[:10])
    print()

Hydrochlorothiazide allergies Allergies: Allergies:
Hydrochlorothiazide

Attending:[**

Abdominal pain chief_complaint Chief Complaint: Chief Complaint:
Abdominal pain

Major Surgical or

stroke history_of_present_illness History of Present Illness: History of Present Illness:
74y female with type

abdominal pain history_of_present_illness History of Present Illness: History of Present Illness:
74y female with type

metastasis history_of_present_illness History of Present Illness: History of Present Illness:
74y female with type

Colon cancer past_medical_history Past Medical History: Past Medical History:
1. Colon cancer dx

hemicolectomy past_medical_history Past Medical History: Past Medical History:
1. Colon cancer dx

XRT past_medical_history Past Medical History: Past Medical History:
1. Colon cancer dx

stroke family_history Family History: Family History:
Mother with stroke at age 82

abdominal pain patient_instructions Discharge Instructions: Discharge Instructions:
Patient ma

# Preprocessing
In preprocessing, we'll take some steps to clean up the text.
- Lower-case (for demonstration purposes only; later steps are sometimes case-sensitive unless explicitly told not to be)
- Replace MIMIC-style time brackets with "2010" and remove all other MIMIC-style formatting
- Replace acronyms such as "dx'd" and "tx'd" to simplify later processing

The preprocessing component is implemented in [nlp_preprocessor](https://github.com/medspacy/nlp_preprocessor).

In [25]:
from medspacy.preprocess import Preprocessor, PreprocessingRule
import re

In [26]:
preprocessor = Preprocessor(nlp.tokenizer)

In [27]:
preprocess_rules = [
    lambda x: x.lower(),
    
    PreprocessingRule(
        re.compile("\[\*\*[\d]{1,4}-[\d]{1,2}(-[\d]{1,2})?\*\*\]"),
        repl="01-01-2010",
        desc="Replace MIMIC date brackets with a generic date."
    ),
    
    PreprocessingRule(
        re.compile("\[\*\*[\d]{4}\*\*\]"),
        repl="2010",
        desc="Replace MIMIC year brackets with a generic year."
    ),
    
    PreprocessingRule(
        re.compile("dx'd"), repl="Diagnosed", 
                  desc="Replace abbreviation"
    ),
    
    PreprocessingRule(
        re.compile("tx'd"), repl="Treated", 
                  desc="Replace abbreviation"
    ),
    
        PreprocessingRule(
        re.compile("\[\*\*[^\]]+\]"), 
        desc="Remove all other bracketed placeholder text from MIMIC"
    )
]

In [28]:
preprocessor.add(preprocess_rules)

In [29]:
nlp.tokenizer = preprocessor

In [30]:
preprocessed_doc = nlp(text)

In [31]:
# Compare the original text with the preprocessed Doc
print(text[:1000])

Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

Service: SURGERY

Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis.

Past Medical History:
1. Colon cancer dx'd in [**2554**], tx'd with hemicolectomy, XRT,
chemo. Last colonoscopy showed: Last CEA was in the 8 range
(down from 9)
2. Type II Diabetes Mellitus
3. Hypertension

Social History:
Married, former tobacco use. No alcohol or drug use.

Family History:
Mother with stroke at age 82. no early deaths.
2 daughters- healthy


Brief Hospital Course:
Ms. [**Known patient lastname 2004**] was admitted on [**2573-5-30**]. Ultrasound

In [32]:
preprocessed_doc[:200]

admission date:  01-01-2010              discharge date:   01-01-2010

date of birth:  01-01-2010             sex:   f

service: surgery

allergies:
hydrochlorothiazide

attending:
chief complaint:
abdominal pain

major surgical or invasive procedure:
picc line 01-01-2010
ercp w/ sphincterotomy 01-01-2010


history of present illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. imaging shows no evidence of metastasis.

past medical history:
1. colon cancer Diagnosed in 2010, Treated with hemicolectomy, xrt,
chemo. last colonoscopy showed: last cea was in the 8 range
(down from 9)
2. type ii diabetes mellitus
3. hypertension

social history:
married, former tobacco use. no alcohol or drug use.

family history:
mother with stroke at age 82. no early deaths

# Postprocessing
The final component we'll introduce is the `postprocessor`. The postprocessor iterates through each entity and checks a series of conditions on each. If all conditions evaluate as `True`, then some action is taken on the entity. Some use cases of this include removing an entity or changing an attributes.

For example, let's say that we want to exclude any entity which comes from the **"patient_instructions"** section, as these are typically not experienced by the patient and are purely hypothetical. We'll write a rule to remove any entity from `doc.ents` if it came from this section. 

The design pattern for a postprocessing rule is as follows:
- A `PostprocessingRule` contains a list of `patterns` and an `action` to take if all of the `patterns` evaluate as `True`
- Each `PostprocessingPattern` takes a `condition`, which evaluates as `True` or `False`. If all patterns return `True`, the action is taken
- Each pattern can take option `condition_args` to pass into the condition check, and each rule takes optional `action_args`
- The module `postprocessing_functions` offer utility functions for the `condition` and `description` arguments

In [33]:
from medspacy.postprocess import Postprocessor, PostprocessingRule, PostprocessingPattern
from medspacy.postprocess import postprocessing_functions

In [34]:
postprocessor = Postprocessor(debug=False) # Set to True for more verbose information about rule matching

In [35]:
nlp.add_pipe(postprocessor)

In [36]:
postprocess_rules = [
    # Instantiate our rule
    PostprocessingRule(
        # Pass in a list of patterns
        patterns=[
            # The pattern will check if the entitie's section is "patient_instructions"
            PostprocessingPattern(condition=lambda ent: ent._.section_title == "patient_instructions"),
        ],
        # If all patterns are True, this entity will be removed.
        action=postprocessing_functions.remove_ent,
        description="Remove any entities from the instructions section."
    ),
    
]

Before adding the postprocessingrules, here are the final 5 entities:

In [37]:
print("Before:")
for ent in doc.ents[-5:]:
    print(ent, ent._.section_title)

Before:
hemicolectomy past_medical_history
XRT past_medical_history
stroke family_history
abdominal pain patient_instructions
abdominal pain patient_instructions


In [38]:
postprocessor.add(postprocess_rules)

In [39]:
doc = nlp(text)

Afterwards, the final 2 entities have been removed:

In [40]:
print("After:")
for ent in doc.ents[-5:]:
    print(ent, ent._.section_title)

After:
metastasis history_of_present_illness
colon cancer past_medical_history
hemicolectomy past_medical_history
xrt past_medical_history
stroke family_history


In [41]:
nlp.pipe_names

['sentencizer', 'target_matcher', 'context', 'sectionizer', 'postprocessor']